In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import DataLoader, Dataset
import cv2
import os
import glob
import random
import itertools
import re

from parse_config import parse_model_config
from models import Darknet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [143]:
LABELLED_DATA_PATH = 'data/train_images/'

def img_transform(img_path, boxes):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    square_dim = max(w,h)
    pad_len = abs(w-h)//2
    pad_dim = (0,0, 0,0, pad_len, pad_len) if w >= h else (0,0, pad_len,pad_len, 0,0)
    img = F.pad(torch.from_numpy(img), pad_dim, )
    shift_x, shift_y = (0, pad_len) if w>=h else (pad_len, 0)
    boxes = torch.IntTensor(boxes)
    boxes[..., :2] += torch.IntTensor([shift_x, shift_y])
    boxes[:, :2] += (boxes[:, 2:]//2)
    boxes = boxes/float(img.shape[0])
    img = F.interpolate(img.permute(2,0,1).unsqueeze(0), size=416, mode='nearest').float().squeeze(0)
    return img, boxes

def draw_boxes(img, box_norm):
    h, w = img.shape[:2]
    square_dim = max(w,h)
    pad_len = abs(w-h)//2
    boxes = (box_norm * w).astype(int)
    shift_x, shift_y = (0, pad_len) if w>=h else (pad_len, 0)
    print(pad_len)
    boxes[:, :2] -= [shift_x, shift_y]
    boxes[:, :2] -= (boxes[:, 2:]//2)
    for x1, y1, w, h in boxes:
        cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (255, 0, 0), 2)

    plt.imshow(img)

class FaceTrainSet(Dataset):
    def __init__(self, data_path, labels_path, transforms=None):
        super(FaceTrainSet, self).__init__()
        self.data_path = data_path
        self.transforms = transforms
        self.labels_path = labels_path

        self.labels = np.loadtxt(self.labels_path, dtype=str, delimiter='\n')
        self.file_names = list(filter(lambda x:x.endswith('.jpg'), self.labels))
        self.file_name_index = [i for i in range(len(self.labels)) if self.labels[i].endswith('.jpg')]
        
        self.batch_count = 0
    
    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, index):
        idx = self.file_name_index[index]
        try:
            label_info = self.labels[idx: self.file_name_index[self.file_name_index.index(idx)+1]]
        except IndexError as e:
            label_info = self.labels[idx:]
        file_name = label_info[0]
        print(file_name)
        num_faces = label_info[1]
        box_dims = [np.array(values.split()).astype(np.int)[:4] for values in label_info[2:]]
        img_tensor, boxes = img_transform(LABELLED_DATA_PATH+file_name, box_dims)
        return (img_tensor, boxes, num_faces)
    
    def collate_fn(self, batch):
        img_tensors, targets, num_faces = list(zip(*batch))
        # Remove empty placeholder targets
        targets = [boxes for boxes in targets if boxes is not None] 
        # Add sample index to targets
        for i, boxes in enumerate(targets):
            index = torch.ones(len(boxes), 1) * i
            targets[i] = torch.cat((index, boxes), 1)
        targets = torch.cat(targets, 0)
        # Selects new image size every tenth batch
        #         if self.multiscale and self.batch_count % 10 == 0:
        #             self.img_size = random.choice(range(self.min_size, self.max_size + 1, 32))
        #         # Resize images to input shape
        #         imgs = torch.stack([resize(img, self.img_size) for img in imgs])
        img_tensors = torch.stack([img for img in img_tensors])
        self.batch_count += 1
        return img_tensors, targets, num_faces
    
def xywh2xyxy(x):
    y = x.new(x.shape)
    y[..., 0] = x[..., 0] - x[..., 2] // 2
    y[..., 1] = x[..., 1] - x[..., 3] // 2
    y[..., 2] = x[..., 0] + x[..., 2] // 2
    y[..., 3] = x[..., 1] + x[..., 3] // 2
    return y

def non_max_suppression(prediction, conf_thres=0.5, nms_thres=0.4):
    """
    Removes detections with lower object confidence score than 'conf_thres' and performs
    Non-Maximum Suppression to further filter detections.
    Returns detections with shape:
        (x1, y1, x2, y2, object_conf, class_score, class_pred)
    """

    # From (center x, center y, width, height) to (x1, y1, x2, y2)
    prediction[..., :4] = xywh2xyxy(prediction[..., :4])
    output = [None for _ in range(len(prediction))]
    for image_i, image_pred in enumerate(prediction):
        # Filter out confidence scores below threshold
        image_pred = image_pred[image_pred[:, 4] >= conf_thres]
        # If none are remaining => process next image
        if not image_pred.size(0):
            continue
        # Object confidence times class confidence
        print(image_pred[:, 4].shape, image_pred[:, 5:].shape)
        score = image_pred[:, 4] * image_pred[:, 5:].max(1)[0]
        # Sort by it
        image_pred = image_pred[(-score).argsort()]
        class_confs, class_preds = image_pred[:, 5:].max(1, keepdim=True)
        detections = torch.cat((image_pred[:, :5], class_confs.float(), class_preds.float()), 1)
        # Perform non-maximum suppression
        keep_boxes = []
        while detections.size(0):
            large_overlap = bbox_iou(detections[0, :4].unsqueeze(0), detections[:, :4]) > nms_thres
            label_match = detections[0, -1] == detections[:, -1]
            # Indices of boxes with lower confidence scores, large IOUs and matching labels
            invalid = large_overlap & label_match
            weights = detections[invalid, 4:5]
            # Merge overlapping bboxes by order of confidence
            detections[0, :4] = (weights * detections[invalid, :4]).sum(0) / weights.sum()
            keep_boxes += [detections[0]]
            detections = detections[~invalid]
        if keep_boxes:
            output[image_i] = torch.stack(keep_boxes)

    return output

In [127]:
dataset = FaceTrainSet('data/train_images/', 'data/labels.txt')
dataloader = DataLoader(dataset, batch_size=2, collate_fn = dataset.collate_fn)

In [128]:
img_tensors, boxes, num_faces = next(iter(dataloader))

58--Hockey/58_Hockey_icehockey_puck_58_399.jpg
58--Hockey/58_Hockey_icehockey_puck_58_286.jpg


In [129]:
model = Darknet('yolov3.cfg')

In [130]:
out= model(img_tensors)

In [138]:
#non_max_suppression(out)

torch.Size([10647, 5])

In [ ]:
out_1 = out[0]
out_2 = out[1]
out_3 = out[2]

In [ ]:
pred_boxes = out_1[..., :4]
pred_boxes.shape



In [ ]:
def bbox_wh_iou(anch_x, anch_y, target_x, target_y):
    inter_area = torch.min(target_x, anch_x) * torch.min(target_y, anch_y)
    union_area = (anch_x * anch_y) + (target_x * target_y) - inter_area
    return inter_area/union_area

def build_targets(pred_boxes, target, anchors, ignore_thres):
    ByteTensor = torch.cuda.ByteTensor if pred_boxes.is_cuda else torch.ByteTensor
    FloatTensor = torch.cuda.FloatTensor if pred_boxes.is_cuda else torch.FloatTensor

    nS = pred_boxes.size(0)
    nA = pred_boxes.size(1)
    nG = pred_boxes.size(2)

    # Output tensors
    obj_mask = ByteTensor(nS, nA, nG, nG).fill_(0)
    noobj_mask = ByteTensor(nS, nA, nG, nG).fill_(1)
    iou_scores = FloatTensor(nS, nA, nG, nG).fill_(0)
    tx = FloatTensor(nS, nA, nG, nG).fill_(0)
    ty = FloatTensor(nS, nA, nG, nG).fill_(0)
    tw = FloatTensor(nS, nA, nG, nG).fill_(0)
    th = FloatTensor(nS, nA, nG, nG).fill_(0)
    target_boxes = target[:, 1:5]
    s = target[:, 0].long()
    g_w, g_h = target_boxes[:, 2:4].T
    g_xy = target_boxes[:, 0:2] * nG
    g_i, g_j = g_xy.long().t()
    g_x, g_y = g_xy.t()
    
    ious = torch.stack([bbox_wh_iou(anch_x, anch_y, g_w, g_h) for anch_x, anch_y in anchors])
    best_ious, ind = ious.max(0)
    
    obj_mask[s, ind, g_i, g_j] = 1
    noobj_mask[s, ind, g_i, g_j] = 0
    
    tx[s, ind, g_i, g_j] = g_x - g_x.floor()
    ty[s, ind, g_i, g_j] = g_y - g_y.floor()
    
    for i, anchor_ious in enumerate(ious.t()):
        noobj_mask[s[i], anchor_ious > ignore_thres, g_i[i], g_j[i]] = 0
    
    tw[s, ind, g_i, g_j] = torch.log(g_w/anchors[ind][:, 0] + 1e-16)
    th[s, ind, g_i, g_j] = torch.log(g_h/anchors[ind][:, 1] + 1e-16)
    
    iou_scores[s, ind, g_i, g_j] = bbox_iou(pred_boxes[s, ind, g_i, g_j], target_boxes,  x1y1x2y2=False)
    
    tconf = obj_mask.float()
    return iou_scores, obj_mask, noobj_mask, tx, ty, tw, th, tconf

In [ ]:
anchors = model.module_defs[-1]['anchors'].split(',')
anchors = [int(x) for x in anchors]
anchors = [(anchors[i]/416/13, anchors[i+1]/416/13) for i in range(0, len(anchors), 2)]
anchors = torch.FloatTensor(anchors[:3])

iou_scores = torch.FloatTensor(2, 3, 13, 13).fill_(0)

In [ ]:
target_boxes = ((boxes[:, 1:5])) * 13
s = boxes[:, 0].long()
target_xy = target_boxes[:, 0:2]
target_wh = target_boxes[:, 2:4]
target_x, target_y = target_xy.long().t()
target_w, target_h = target_wh.T

In [ ]:
def bbox_wh_iou(anch_x, anch_y, target_x, target_y):
    inter_area = torch.min(target_x, anch_x) * torch.min(target_y, anch_y)
    union_area = (anch_x * anch_y) + (target_x * target_y) - inter_area
    return inter_area/union_area

In [ ]:
ious = torch.stack([bbox_wh_iou(anch_x, anch_y, target_w, target_h) for anch_x, anch_y in anchors])

In [ ]:
best_ious, ind = ious.max(0)

In [190]:
def bbox_iou(boxes1, boxes2, x1y1x2y2=False):
    if not x1y1x2y2:
        boxes1_x1, boxes1_x2 = boxes1[:,0] - boxes1[:,2]/2, boxes1[:,0] + boxes1[:,2]/2
        boxes1_y1, boxes1_y2 = boxes1[:,1] - boxes1[:,3]/2, boxes1[:,1]+boxes1[:,3]/2
        boxes2_x1, boxes2_x2 = boxes2[:,0] - boxes2[:,2]/2, boxes2[:,0] + boxes2[:,2]/2
        boxes2_y1, boxes2_y2 = boxes2[:,1] - boxes2[:,3]/2, boxes2[:,1]+boxes2[:,3]/2
    
    else:
        boxes1_x1, boxes1_x2, boxes1_y1, boxes1_y2 = boxes1[:, 0], boxes1[:, 1], boxes1[:, 2], boxes1[:, 3]
        boxes2_x1, boxes2_x2, boxes2_y1, boxes2_y2 = boxes2[:, 0], boxes2[:, 1], boxes2[:, 2], boxes2[:, 3]
        
    inter_w =  torch.clamp(torch.min(boxes1_x2, boxes2_x2) - torch.max(boxes1_x1, boxes2_x1) + 1, min=0)
    inter_h = torch.clamp(torch.min(boxes1_y2, boxes2_y2) - torch.max(boxes1_y1, boxes2_y1 ) + 1, min=0)
    
    inter_area = inter_w * inter_h
    
    boxes1_area = (boxes1_x2-boxes1_x1+1) * (boxes1_y2-boxes1_y1+1)
    boxes2_area = (boxes2_x2-boxes2_x1+1) * (boxes2_y2-boxes2_y1+1)
    
    iou = inter_area/(boxes1_area + boxes2_area - inter_area+1e-16)
    
    return iou

In [ ]:
boxes1 = boxes[:, 1:]
boxes2 = torch.cat([boxes1, boxes1+0.01])

In [ ]:
img = cv2.imread('data/train_images/58--Hockey/58_Hockey_icehockey_puck_58_399.jpg')
draw_boxes(img, boxes2.numpy())

In [ ]:
try: 
    with open ('config.yml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print(e)

In [ ]:
from __future__ import division

from models import *
#from utils.logger import *
#from utils.utils import *
from utils import *
from data_manager import *
from parse_config import *
#from test import evaluate

from terminaltables import AsciiTable

import os
import sys
import time
import datetime
import argparse
import yaml

import torch
#from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

# parser = argparse.ArgumentParser()
# parser.add_argument("--config", type=str, default='config.yml', help="configuration file in yml format")
# opt = parser.parse_args()

#config_file = opt.config
try: 
    with open ('config.yml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print("unable to read config file")

model_config = config['train']['model']
data_config = config['train']['data']
train_params = config['train']['params']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs("output", exist_ok=True)
os.makedirs("checkpoints4", exist_ok=True)

#writer = SummaryWriter()

# Get data configuration

train_labels = data_config["train_labels"]
valid_labels = data_config["valid_labels"]
img_path = data_config["img_path"]
#class_names = load_classes(data_config["class_names"])

# Initiate model
model = Darknet(model_config["model_def"]).to(device)
model.apply(weights_init_normal)

# If specified we start from checkpoint
if model_config['pretrained_weights']:
    if model_config['pretrained_weights'].endswith(".pth"):
        model.load_state_dict(torch.load(model_config["pretrained_weights"]))
    else:
        model.load_darknet_weights(model_config["pretrained_weights"])

# Get dataloader
dataset = FaceTrainSet(img_path, train_labels)
dataloader = torch.utils.data.DataLoader(dataset, train_params["batch_size"], shuffle=True,num_workers=train_params["n_cpu"] ,
            collate_fn = dataset.collate_fn)
optimizer = torch.optim.Adam(model.parameters())

metrics = [
    "grid_size",
    "loss",
    "x",
    "y",
    "w",
    "h",
    "conf",
    "conf_obj",
    "conf_noobj",
]

for epoch in range(train_params["epochs"]):
    model.train()
    start_time = time.time()
    for batch_i, (imgs, targets, _) in enumerate(dataloader):
        batches_done = len(dataloader) * epoch + batch_i

        imgs = Variable(imgs.to(device))
        targets = Variable(targets.to(device), requires_grad=False)

        loss, outputs = model(imgs, targets)
        loss.backward()

        if batches_done % train_params["gradient_accumulations"]:
            # Accumulates gradient before each step
            optimizer.step()
            optimizer.zero_grad()

        # ----------------
        #   Log progress
        # ----------------

In [ ]:
dataset = FaceTrainSet(img_path, train_labels)
dataloader = torch.utils.data.DataLoader(dataset, train_params["batch_size"])

In [ ]:
img_path

In [ ]:
type(img)

In [218]:
def non_max_supression(predictions, conf_thres=0.5, nms_thres=0.4):
    output = [None for _ in range(len(predictions))]
    for i , img_pred in enumerate(predictions):
        img_pred = img_pred[img_pred[:, 4] > conf_thres]
        img_pred[:, :4] = x1y1x2y2(img_pred[:, :4])
        if not img_pred.shape[0]:
            continue
        keep_boxes = []
        while img_pred.size(0):
            overlap_iou = bbox_iou(img_pred[0, :4].unsqueeze(0), img_pred[:, :4])
            
            invalid_iou = overlap_iou < nms_thres
            valid_boxes= img_pred[~invalid_iou]
            if valid_boxes.size(0):
                weights = valid_boxes[:, 4:5]
                box = torch.sum(valid_boxes[:, :4] * weights, axis = 0)/torch.sum(weights)
                keep_boxes.append(box)
            img_pred = img_pred[invalid_iou]
        
        output[i] = keep_boxes
    return output



In [219]:
arr = torch.randint(1,10,(2, 49*3, 5)).float()
#arr1 = arr[0][arr[0][:, 4] > 2]

In [217]:
def x1y1x2y2(boxes):
    x1, x2 = boxes[:,0] - boxes[:,2]/2, boxes[:,0] + boxes[:,2]/2
    y1, y2 = boxes[:,1] - boxes[:,3]/2, boxes[:,1]+boxes[:,3]/2
    return torch.stack((x1,y1,x2,y2), 1)

In [222]:
if True:
    a = 2
print(a)

2
